tobac example: Compute bulk statistics as a postprocessing step
=== 

Instead of during the feature detection or segmentation process, you can also calculate bulk statistics of your detected/tracked objects as a postprocessing step, i.e. based on a segmentation mask. This makes it possible to combine different datasets and derive statistics for your detected features based on other input fields (e.g., get precipitation statistics under cloud features that were segmented based on brightness temperatures or outgoing longwave radiation). 

This notebook shows an example for how to read in a segmentation mask and input data with the same grid spacing and dimensions to compute statistics for the detected features. We use here the segmentation mask and data from [our example for precipitation tracking](https://github.com/tobac-project/tobac/blob/main/examples/Example_Precip_Tracking/Example_Precip_Tracking.ipyn) (see also ...). 

In [1]:
# Import libraries
import iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import iris.plot as iplt
import iris.quickplot as qplt
import datetime
import shutil
from six.moves import urllib
from pathlib import Path
%matplotlib inline

In [2]:
# Import tobac itself
import tobac
print('using tobac version', str(tobac.__version__))

using tobac version 1.5.0


In [3]:
# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [4]:
# read in mask file from segmentation process 
savedir=Path("Save")
Features_Precip = iris.load_cube(savedir / 'Mask_Segmentation_precip.nc')
Precip = iris.load_cube('../climate-processes-tobac_example_data-b3e69ee/data/Example_input_Precip.nc')

In [5]:
import xarray as xr 
Mask_Precip = xr.open_dataset(savedir / 'Mask_Segmentation_precip.nc').segmentation_mask
Features_Precip = pd.read_hdf(savedir / 'Features_Precip.h5', 'table')
Precip = xr.open_dataset('../climate-processes-tobac_example_data-b3e69ee/data/Example_input_Precip.nc').surface_precipitation_average

**Get bulk statistics from mask file**

You can decide which statistics to calculate by providing a dictionary with the name of the metric as keys (this will be the name of the column added to the dataframe) and functions as values. Note that it is also possible to provide input parameter to these functions. 

In [6]:
from tobac.utils.general import get_statistics_from_mask

#### Defining the dictionary for the statistics to be calculated 

In [7]:
statistics = {}
statistics['mean_precip'] = np.mean
statistics['total_precip'] = np.sum
statistics['max_precip'] = np.max

For some functions, we need to provide additional input parameters, e.g. [np.percentile()](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html). These can be provided as key word arguments in form of a dictionary. So instead of the function, you can provide a tuple with both the function and its respective input parameters: 


In [8]:
statistics['percentiles'] = (np.percentile, {'q': [95,99]})

In [24]:
Mask_Precip = iris.load_cube(savedir / 'Mask_Segmentation_precip.nc', 'segmentation_mask')
Precip = iris.load_cube('../climate-processes-tobac_example_data-b3e69ee/data/Example_input_Precip.nc', 'surface_precipitation_average')

In [26]:
Precip

Surface Precipitation Average (mm h-1),time,south_north,west_east
Shape,47,198,198
Dimension coordinates,,,
time,x,-,-
south_north,-,x,-
west_east,-,-,x
Auxiliary coordinates,,,
projection_y_coordinate,-,x,-
y,-,x,-
latitude,-,x,x
longitude,-,x,x


In [27]:
features_with_stats = get_statistics_from_mask(Mask_Precip, Precip, features = Features_Precip, func_dict = statistics)

ValueError: Arrays chunk sizes are unknown. 

A possible solution: https://docs.dask.org/en/latest/array-chunks.html#unknown-chunks
Summary: to compute chunks sizes, use

   x.compute_chunk_sizes()  # for Dask Array `x`
   ddf.to_dask_array(lengths=True)  # for Dask DataFrame `ddf`

### Look at the output: 

In [10]:
features_with_stats.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,mean_precip,total_precip,max_precip,percentiles,...,timestr,south_north,west_east,projection_y_coordinate,y,latitude,longitude,projection_x_coordinate,x,ncells
0,0,1,50.065727,139.857477,9,1,1.629695,16.296951,2.289786,"([2.221776068210602, 2.276183712482452],)",...,2013-06-19 20:05:00,331.065727,420.857477,165782.863285,331.065727,29.846362,-94.172015,210678.738492,420.857477,10
1,0,15,120.527119,172.500325,4,1,1.409547,14.095468,1.819811,"([1.8030404090881347, 1.8164567756652832],)",...,2013-06-19 20:05:00,401.527119,453.500325,201013.559414,401.527119,30.166929,-93.996892,227000.162468,453.500325,10
2,0,18,126.779273,145.368401,15,1,2.441526,26.856783,3.771701,"([3.710712432861328, 3.759503173828125],)",...,2013-06-19 20:05:00,407.779273,426.368401,204139.636582,407.779273,30.196499,-94.139960,213434.200454,426.368401,11
3,0,34,111.611369,155.452030,4,2,1.938501,36.831512,4.067666,"([3.940941762924194, 4.042321195602417],)",...,2013-06-19 20:05:00,392.611369,436.452030,196555.684682,392.611369,30.126871,-94.087317,218476.015240,436.452030,19
4,0,35,111.765231,164.938866,8,2,2.486886,49.737709,4.380943,"([4.087516045570374, 4.3222578477859495],)",...,2013-06-19 20:05:00,392.765231,445.938866,196632.615461,392.765231,30.127221,-94.037226,223219.433218,445.938866,20
